# Generating a list of MAL users
* We start with a list of starting usernames, which can either be stored in `data/user_facts/queue.txt` or specified in the notebook
* Then, we do a bread-first search of their friends until the friend graph is spanned
* You can terminate or restart the notebook at any point. All users found so far will be stored at `data/user_facts/usernames.txt` 
* This notebook may run indefinitely. Feel free to manually terminate once an acceptable number of users have been found

In [1]:
import os
import time

import pandas as pd
import requests
from tqdm import tqdm
from ratelimit import limits, sleep_and_retry

In [2]:
data_path = "../../data/user_facts"
if not os.path.exists(data_path):
    os.mkdir(data_path)
os.chdir(data_path)

In [3]:
# reasonable defaults if this is the first time running the notebook
queue = ['Fro116']
usernames = set()
closed_nodes = set()

# if we rerunning the notebook, then resume execution where we last left off
if os.path.exists("queue.txt"):
    with open('queue.txt') as f:
        queue = [x.strip() for x in f.readlines()]   
if os.path.exists("usernames.txt"):
    with open('usernames.txt') as f:
        usernames = {x.strip() for x in f.readlines()}
if os.path.exists("closed_nodes.txt"):
    with open('closed_nodes.txt') as f:
        closed_nodes = {x.strip() for x in f.readlines()} 
open_nodes = set(queue)    

print(f"Starting with {len(queue)} users in queue and {len(closed_nodes)} processed users!")

Starting with 63513 users in queue and 3369 processed users!


In [4]:
@sleep_and_retry
@limits(calls=1, period=4)
def call_api(url):
    response = requests.get(
        url
    )
    return response

In [5]:
from ratelimit import limits, sleep_and_retry


In [6]:
def get_friends(username):
    friends = set()
    
    max_items_per_page = 300  # property of the API
    more_pages = True
    page = 1
    while more_pages:
        response = call_api(f"https://api.jikan.moe/v3/user/{username}/friends/{page}")
        if not response.ok:
            print(f"error: no friends found for {username}. This can happen in rare circumstances")
            return friends
        new_friends = {x['username'] for x in response.json()['friends']}
        friends |= new_friends
        page += 1
        if len(new_friends) < max_items_per_page:
            more_pages = False
    return friends

In [ ]:
# we use a generator for profiling with tqdm
def generator(queue):
    while queue:
        yield

for _ in tqdm(generator(queue)):
    username = queue[0]
    queue = queue[1:]
    friends = get_friends(username)
    
    usernames |= friends
    new_friends = [x for x in friends if x not in closed_nodes and x not in open_nodes]
    queue += new_friends
    open_nodes |= set(new_friends)
    open_nodes.remove(username)    
    closed_nodes.add(username)
    
    pd.Series(queue).to_csv("queue.txt", header=False, index=False)
    pd.Series(sorted(list(usernames))).to_csv("usernames.txt", header=False, index=False)
    pd.Series(sorted(list(closed_nodes))).to_csv("closed_nodes.txt", header=False, index=False)

878it [58:40,  3.92s/it]

error: no friends found for sunmd


7768it [8:39:30,  4.00s/it]

error: no friends found for FeraPlayz


8508it [9:29:03,  4.09s/it]

error: no friends found for MobJAMbatista


11141it [12:25:10,  3.92s/it]

In [ ]:
len(usernames), len(open_nodes), len(closed_nodes), len(queue)